Kindly download the required datasets from the links provided below:

1. **Capital Letters Dataset**  
This dataset contains images of uppercase alphabet characters. It will be used for training and evaluating models that recognize capital letters.  
**Download Link:** [Capital Letters Dataset](https://drive.google.com/file/d/1UX1AdYPRxA5DNU1fzEoUBFi1cbKlf-ij/view?usp=sharing)

2. **Small Letters Dataset**  
This dataset includes images of lowercase alphabet characters and is essential for tasks involving recognition of small letters.  
**Download Link:** [Small Letters Dataset](https://drive.google.com/file/d/1__1374Zr0h66LkElXTaL-g8I_v2F18yb/view?usp=sharing)

Please ensure both datasets are downloaded and saved in the current directory before proceeding with the next steps.

In [1]:
import zipfile

with zipfile.ZipFile("capital.zip", 'r') as zip_ref:
    zip_ref.extractall(".")  # "." indicates the current directory

In [2]:
# Extracting the second zip file
with zipfile.ZipFile("small.zip", 'r') as zip_ref:
    zip_ref.extractall(".")  # "." indicates the current directory

### Consolidating Character Images into a Unified Training Directory

In [3]:
import os
import shutil

# Define the source and destination directories
source_dir = "capital"
destination_dir = "training_data_gan"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Walk through all folders and subfolders inside the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        # Check if the file is an image based on extension
        if file.lower().endswith('.png'):
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_dir, file)

            # Handle duplicate filenames by renaming
            counter = 1
            while os.path.exists(destination_file_path):
                filename, ext = os.path.splitext(file)
                new_filename = f"{filename}_{counter}{ext}"
                destination_file_path = os.path.join(destination_dir, new_filename)
                counter += 1

            shutil.copy2(source_file_path, destination_file_path)

print(f"All images have been copied to {destination_dir}.")

All images have been copied to training_data_gan.


In [4]:
# Define the source and destination directories
source_dir = "small"
destination_dir = "training_data_gan"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Walk through all folders and subfolders inside the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        # Check if the file is an image based on extension
        if file.lower().endswith('.png'):
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_dir, file)

            # Handle duplicate filenames by renaming
            counter = 1
            while os.path.exists(destination_file_path):
                filename, ext = os.path.splitext(file)
                new_filename = f"{filename}_{counter}{ext}"
                destination_file_path = os.path.join(destination_dir, new_filename)
                counter += 1

            shutil.copy2(source_file_path, destination_file_path)

print(f"All images have been copied to {destination_dir}.")

All images have been copied to training_data_gan.


### Padding and Resizing Word Images to Uniform 200×200 Dimensions

In [5]:
import os
from PIL import Image, ImageOps

# Adding padding to make images square and resizing to 200x200
def pad_and_resize_images(folder_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        raise ValueError(f"The folder {folder_path} does not exist")

    # Defining the target aspect ratio and size
    target_aspect_ratio = 1  # 1:1 aspect ratio
    target_width = 200
    target_height = 200

    # Iterating through all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                # Open the image
                with Image.open(file_path) as img:
                    img = img.convert('L')
                    width, height = img.size
                    aspect_ratio = width / height

                    # Add padding to make the image square
                    if aspect_ratio < target_aspect_ratio:
                        # Height > Width, so pad left and right
                        padding = (height - width) // 2
                        padded_img = ImageOps.expand(img, border=(padding, 0, padding, 0), fill='white')
                    elif aspect_ratio > target_aspect_ratio:
                        # Width > Height, so pad top and bottom
                        padding = (width - height) // 2
                        padded_img = ImageOps.expand(img, border=(0, padding, 0, padding), fill='white')
                    else:
                        padded_img = img  # Already square

                    # Resize the image to 200x200
                    resized_img = padded_img.resize((target_width, target_height))

                    # Save the processed image back to the original path
                    resized_img.save(file_path)

                    print(f"Processed and replaced: {file_path}")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

training_data = "training_data_gan" # Please provide your image folder path here
pad_and_resize_images(training_data)


Processed and replaced: training_data_gan\& (1).png
Processed and replaced: training_data_gan\& (13).png
Processed and replaced: training_data_gan\& (14).png
Processed and replaced: training_data_gan\& (15).png
Processed and replaced: training_data_gan\& (17).png
Processed and replaced: training_data_gan\& (18).png
Processed and replaced: training_data_gan\& (19).png
Processed and replaced: training_data_gan\& (20).png
Processed and replaced: training_data_gan\& (25).png
Processed and replaced: training_data_gan\& (26).png
Processed and replaced: training_data_gan\& (27).png
Processed and replaced: training_data_gan\& (28).png
Processed and replaced: training_data_gan\& (29).png
Processed and replaced: training_data_gan\& (30).png
Processed and replaced: training_data_gan\& (31).png
Processed and replaced: training_data_gan\& (32).png
Processed and replaced: training_data_gan\& (33).png
Processed and replaced: training_data_gan\& (34).png
Processed and replaced: training_data_gan\& (35

### Applying Rotation-Based Data Augmentation to Word Images

In [6]:
import numpy as np
from PIL import Image
import os

def rotation_aug(training_data):
    # Loop through all files in the input folder
    for filename in os.listdir(training_data):
        if filename.endswith('.png'):
            # Open an image file
            with Image.open(os.path.join(training_data, filename)) as img:
                # Loop from -2 to +2 degrees
                for angle in range(-2, 3):
                    if(angle==0):
                        continue
                    # Rotate the image
                    rotated_img = img.rotate(angle, expand=True)
                    # Construct the output filename
                    new_filename = f"{os.path.splitext(filename)[0]}_rot{angle}{os.path.splitext(filename)[1]}"
                    # Save the rotated image to the output folder
                    rotated_img.save(os.path.join(training_data, new_filename))

training_data = "training_data_gan"
rotation_aug(training_data)
print("Image augmentation by Rotation completed.")

Image augmentation by Rotation completed.


### Uniform Resizing of Word Images to 200×200 Pixels

In [7]:
# Resizing images to 200x200 pixels
def resize_images_in_folder(input_folder, new_size=(200,200)):
    # Looping through all files in the input folder
    for filename in os.listdir(input_folder):
        # Opening the image
        with Image.open(os.path.join(input_folder, filename)) as img:
            # Resizing the image
            resized_img = img.resize(new_size)
            # Saving the resized image to the output folder
            output_filename = os.path.splitext(filename)[0] + '.png'  # Ensure output format is PNG
            resized_img.save(os.path.join(input_folder, output_filename))

training_image_dir = "training_data_gan"
resize_images_in_folder(training_image_dir)